In [24]:
# Setup - Run this cell first
import polars as pl

# Load data
airlines = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airlines.csv')
airports = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airports.csv', null_values=["NA"])
flights = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_flights.csv', null_values=["NA"])
planes = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_planes.csv', null_values=["NA"])
weather = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_weather.csv', null_values=["NA"], infer_schema_length=10000)

flights = flights.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))
weather = weather.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))

# Create SQL context
ctx = pl.SQLContext(
    airlines=airlines,
    airports=airports,
    flights=flights,
    planes=planes,
    weather=weather,
    eager_execution=True
)

print("Setup complete! Tables available:")
print(ctx.execute("SHOW TABLES"))
print("Weather:", weather.columns)
print("Flights:", flights.columns)
print("Planes:", planes.columns)
print("Airlines:", airlines.columns)
print("Airports:", airports.columns)

Setup complete! Tables available:
shape: (5, 1)
┌──────────┐
│ name     │
│ ---      │
│ str      │
╞══════════╡
│ airlines │
│ airports │
│ flights  │
│ planes   │
│ weather  │
└──────────┘
Weather: ['origin', 'year', 'month', 'day', 'hour', 'temp', 'dewp', 'humid', 'wind_dir', 'wind_speed', 'wind_gust', 'precip', 'pressure', 'visib', 'time_hour']
Flights: ['year', 'month', 'day', 'dep_time', 'sched_dep_time', 'dep_delay', 'arr_time', 'sched_arr_time', 'arr_delay', 'carrier', 'flight', 'tailnum', 'origin', 'dest', 'air_time', 'distance', 'hour', 'minute', 'time_hour']
Planes: ['tailnum', 'year', 'type', 'manufacturer', 'model', 'engines', 'seats', 'speed', 'engine']
Airlines: ['carrier', 'name']
Airports: ['faa', 'name', 'lat', 'lon', 'alt', 'tz', 'dst', 'tzone']


/tmp/ipython-input-3186420141.py:15: DeprecationWarning: The argument `eager_execution` for `SQLContext.__init__` is deprecated. It has been renamed to `eager`.
  ctx = pl.SQLContext(


In [27]:
#Exercise 1
#q1.1
result = ctx.execute("""
select DISTINCT carrier
from airlines
""")
print(result)

shape: (16, 1)
┌─────────┐
│ carrier │
│ ---     │
│ str     │
╞═════════╡
│ 9E      │
│ AA      │
│ AS      │
│ B6      │
│ DL      │
│ …       │
│ UA      │
│ US      │
│ VX      │
│ WN      │
│ YV      │
└─────────┘


In [31]:
#q1.2
result = ctx.execute("""
select dest, flight
from flights
group by dest, flight
order by flight desc
limit 10
""")

print(result)

shape: (10, 2)
┌──────┬────────┐
│ dest ┆ flight │
│ ---  ┆ ---    │
│ str  ┆ i64    │
╞══════╪════════╡
│ ORD  ┆ 8500   │
│ IAD  ┆ 6181   │
│ SDF  ┆ 6181   │
│ IAD  ┆ 6180   │
│ IAD  ┆ 6177   │
│ IAD  ┆ 6171   │
│ MSP  ┆ 6168   │
│ IAD  ┆ 6167   │
│ ROC  ┆ 6167   │
│ IAD  ┆ 6165   │
└──────┴────────┘


In [35]:
#q1.3
result = ctx.execute("""
select flight, dep_delay
from flights
where dep_delay > 120
group by flight,dep_delay
order by dep_delay desc
""")

print(result)

shape: (9_401, 2)
┌────────┬───────────┐
│ flight ┆ dep_delay │
│ ---    ┆ ---       │
│ i64    ┆ i64       │
╞════════╪═══════════╡
│ 51     ┆ 1301      │
│ 3535   ┆ 1137      │
│ 3695   ┆ 1126      │
│ 177    ┆ 1014      │
│ 3075   ┆ 1005      │
│ …      ┆ …         │
│ 3315   ┆ 121       │
│ 1831   ┆ 121       │
│ 4536   ┆ 121       │
│ 1465   ┆ 121       │
│ 4419   ┆ 121       │
└────────┴───────────┘


In [40]:
#Exercise 2
#q2.1
result = ctx.execute("""
select origin, avg(dep_delay) as avg_dep_delay
from flights
where dep_delay is not null
group by origin
order by avg_dep_delay desc
""")
print(result)

shape: (3, 2)
┌────────┬───────────────┐
│ origin ┆ avg_dep_delay │
│ ---    ┆ ---           │
│ str    ┆ f64           │
╞════════╪═══════════════╡
│ EWR    ┆ 15.107954     │
│ JFK    ┆ 12.112159     │
│ LGA    ┆ 10.346876     │
└────────┴───────────────┘


In [54]:
#q2.2
result = ctx.execute("""
select month, count(*) AS flight_count
from flights
group by month
order by flight_count DESC
limit 1
""")
print(result)

shape: (1, 2)
┌───────┬──────────────┐
│ month ┆ flight_count │
│ ---   ┆ ---          │
│ i64   ┆ u32          │
╞═══════╪══════════════╡
│ 7     ┆ 29425        │
└───────┴──────────────┘


In [58]:
#2.3

result = ctx.execute("""
select
    carrier,
    sum(case when dep_delay <= 15 then 1 else 0 end) * 100 / count(dep_delay) as on_time_percentage
from flights
where dep_delay is not null
group by carrier
order by on_time_percentage DESC
""")

print(result)

shape: (16, 2)
┌─────────┬────────────────────┐
│ carrier ┆ on_time_percentage │
│ ---     ┆ ---                │
│ str     ┆ u32                │
╞═════════╪════════════════════╡
│ HA      ┆ 92                 │
│ US      ┆ 87                 │
│ AS      ┆ 86                 │
│ AA      ┆ 84                 │
│ DL      ┆ 83                 │
│ …       ┆ …                  │
│ FL      ┆ 73                 │
│ WN      ┆ 73                 │
│ F9      ┆ 71                 │
│ YV      ┆ 71                 │
│ EV      ┆ 69                 │
└─────────┴────────────────────┘


In [63]:
#Exercise 3
#q3.1
print(flights.columns)
print(airlines.columns)

result = ctx.execute("""
select a.carrier as carrier_code, a.name as airline_name, f.flight as flight_number , f.origin, f.dest
from flights f
left join airlines a on f.carrier = a.carrier
limit 20
""")
print(result)

['year', 'month', 'day', 'dep_time', 'sched_dep_time', 'dep_delay', 'arr_time', 'sched_arr_time', 'arr_delay', 'carrier', 'flight', 'tailnum', 'origin', 'dest', 'air_time', 'distance', 'hour', 'minute', 'time_hour']
['carrier', 'name']
shape: (20, 5)
┌──────────────┬────────────────────────┬───────────────┬────────┬──────┐
│ carrier_code ┆ airline_name           ┆ flight_number ┆ origin ┆ dest │
│ ---          ┆ ---                    ┆ ---           ┆ ---    ┆ ---  │
│ str          ┆ str                    ┆ i64           ┆ str    ┆ str  │
╞══════════════╪════════════════════════╪═══════════════╪════════╪══════╡
│ UA           ┆ United Air Lines Inc.  ┆ 1545          ┆ EWR    ┆ IAH  │
│ UA           ┆ United Air Lines Inc.  ┆ 1714          ┆ LGA    ┆ IAH  │
│ AA           ┆ American Airlines Inc. ┆ 1141          ┆ JFK    ┆ MIA  │
│ B6           ┆ JetBlue Airways        ┆ 725           ┆ JFK    ┆ BQN  │
│ DL           ┆ Delta Air Lines Inc.   ┆ 461           ┆ LGA    ┆ ATL  │
│ …      

In [69]:
#q3.2
result = ctx.execute("""
select
    a.name as airline_name,
    avg(2013 - p.year) as average_plane_age
from flights as f
join planes as p on f.tailnum = p.tailnum
join airlines as a on f.carrier = a.carrier
where f.year = 2013 and p.year is not null
group by a.name
order by average_plane_age desc
""")

print(result)

shape: (16, 2)
┌─────────────────────────────┬───────────────────┐
│ airline_name                ┆ average_plane_age │
│ ---                         ┆ ---               │
│ str                         ┆ f64               │
╞═════════════════════════════╪═══════════════════╡
│ Envoy Air                   ┆ 35.319            │
│ American Airlines Inc.      ┆ 25.869426         │
│ Delta Air Lines Inc.        ┆ 16.372169         │
│ United Air Lines Inc.       ┆ 13.207691         │
│ AirTran Airways Corporation ┆ 11.385829         │
│ …                           ┆ …                 │
│ JetBlue Airways             ┆ 6.686702          │
│ Frontier Airlines Inc.      ┆ 4.87874           │
│ Virgin America              ┆ 4.473643          │
│ Alaska Airlines Inc.        ┆ 3.33662           │
│ Hawaiian Airlines Inc.      ┆ 1.548387          │
└─────────────────────────────┴───────────────────┘


In [73]:
#q3.3
result = ctx.execute("""
select
    f.flight,
    f.dep_delay,
    w.wind_speed,
    w.precip,
    f.origin
from flights as f
join weather as w
  on f.time_hour = w.time_hour and f.origin = w.origin
where f.dep_delay > 30 and (w.wind_speed > 20 or w.precip > 0.1)
""")

print(result)

shape: (4_938, 5)
┌────────┬───────────┬────────────┬────────┬────────┐
│ flight ┆ dep_delay ┆ wind_speed ┆ precip ┆ origin │
│ ---    ┆ ---       ┆ ---        ┆ ---    ┆ ---    │
│ i64    ┆ i64       ┆ f64        ┆ f64    ┆ str    │
╞════════╪═══════════╪════════════╪════════╪════════╡
│ 21     ┆ 47        ┆ 21.86482   ┆ 0.0    ┆ JFK    │
│ 199    ┆ 116       ┆ 21.86482   ┆ 0.0    ┆ JFK    │
│ 4090   ┆ 39        ┆ 20.71404   ┆ 0.0    ┆ EWR    │
│ 4231   ┆ 40        ┆ 24.16638   ┆ 0.0    ┆ EWR    │
│ 1010   ┆ 33        ┆ 20.71404   ┆ 0.0    ┆ JFK    │
│ …      ┆ …         ┆ …          ┆ …      ┆ …      │
│ 2347   ┆ 41        ┆ 20.71404   ┆ 0.0    ┆ LGA    │
│ 5503   ┆ 97        ┆ 20.71404   ┆ 0.0    ┆ LGA    │
│ 135    ┆ 42        ┆ 21.86482   ┆ 0.0    ┆ JFK    │
│ 797    ┆ 46        ┆ 21.86482   ┆ 0.0    ┆ JFK    │
│ 985    ┆ 45        ┆ 21.86482   ┆ 0.0    ┆ JFK    │
└────────┴───────────┴────────────┴────────┴────────┘


In [78]:
#Exercise 4
#q4.1
result = ctx.execute("""
select p.manufacturer, p.model, count(*) as number_of_flights
from flights as f
join planes as p on f.tailnum = p.tailnum
group by p.manufacturer, p.model
order by number_of_flights desc
limit 10
""")

print(result)

shape: (10, 3)
┌───────────────────────────────┬─────────────────┬───────────────────┐
│ manufacturer                  ┆ model           ┆ number_of_flights │
│ ---                           ┆ ---             ┆ ---               │
│ str                           ┆ str             ┆ u32               │
╞═══════════════════════════════╪═════════════════╪═══════════════════╡
│ AIRBUS                        ┆ A320-232        ┆ 31278             │
│ EMBRAER                       ┆ EMB-145LR       ┆ 28027             │
│ EMBRAER                       ┆ ERJ 190-100 IGW ┆ 23716             │
│ AIRBUS INDUSTRIE              ┆ A320-232        ┆ 14553             │
│ EMBRAER                       ┆ EMB-145XR       ┆ 14051             │
│ BOEING                        ┆ 737-824         ┆ 13809             │
│ BOMBARDIER INC                ┆ CL-600-2D24     ┆ 11807             │
│ BOEING                        ┆ 737-7H4         ┆ 10389             │
│ BOEING                        ┆ 757-222        

In [201]:
#q4.2
flight_analysis = ctx.execute("""
select *
from flights as f
where f.dest is not null
""")

origin_data = (
    flight_analysis
    .join(
        airports.select(['faa', 'name']),
        left_on='origin',
        right_on='faa',
        how='left'
    ).rename({'name': 'origin_airport'})
)

destination_data = (
    origin_data
    .join(
        airports.select(['faa', 'name']),
        left_on='dest',
        right_on='faa',
        how='left'
    ).rename({'name': 'destination_airport'})
)

performance_data = destination_data.group_by(['origin_airport', 'destination_airport']).agg([
    pl.len().alias('total_flights'),
    pl.col('dep_delay').mean().alias('avg_dep_delay'),
    ((pl.col('dep_delay') > 30).sum() / pl.len() * 100).alias('percentage_delay_over_30')
])

route_performance = performance_data.filter(pl.col('destination_airport') != 'null').sort('total_flights', descending=True).limit(10)

print(route_performance)


shape: (10, 5)
┌─────────────────────┬──────────────────────┬───────────────┬───────────────┬─────────────────────┐
│ origin_airport      ┆ destination_airport  ┆ total_flights ┆ avg_dep_delay ┆ percentage_delay_ov │
│ ---                 ┆ ---                  ┆ ---           ┆ ---           ┆ er_30               │
│ str                 ┆ str                  ┆ u32           ┆ f64           ┆ ---                 │
│                     ┆                      ┆               ┆               ┆ f64                 │
╞═════════════════════╪══════════════════════╪═══════════════╪═══════════════╪═════════════════════╡
│ John F Kennedy Intl ┆ Los Angeles Intl     ┆ 11262         ┆ 8.522508      ┆ 9.829515            │
│ La Guardia          ┆ Hartsfield Jackson   ┆ 10263         ┆ 11.448621     ┆ 12.247881           │
│                     ┆ Atlanta Intl         ┆               ┆               ┆                     │
│ La Guardia          ┆ Chicago Ohare Intl   ┆ 8857          ┆ 10.740758    

In [136]:
#Bonus
#q2.2 using polars
busiest_month = flights.group_by("month").agg(
    pl.len().alias("flight_count")
).sort("flight_count", descending=True).limit(1)

print(busiest_month)

shape: (1, 2)
┌───────┬──────────────┐
│ month ┆ flight_count │
│ ---   ┆ ---          │
│ i64   ┆ u32          │
╞═══════╪══════════════╡
│ 7     ┆ 29425        │
└───────┴──────────────┘
